In [1]:
%%time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Order 66
%config Completer.use_jedi=False
# Ease
import sys
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", 70)
sys.path.append("K:\\Projects\\vscProject\\Kaggle\\HousePrices")

Wall time: 2.45 s


In [2]:
%%time
# Coustom Data Module
from Script.data import data

X_train, y_train, X_test, y_samp = data()

Tokenization took: 0.00 ms
Type conversion took: 9.69 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 10.21 ms
Type conversion took: 10.04 ms
Parser memory cleanup took: 0.00 ms
Tokenization took: 9.82 ms
Type conversion took: 9.99 ms
Parser memory cleanup took: 0.00 ms
Wall time: 3.02 s


# sk-learn Imports

In [3]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, RepeatedKFold
from sklearn.tree import DecisionTreeRegressor, ExtraTreeRegressor, plot_tree

In [4]:
cv = RepeatedKFold(random_state=3007)

# Decision Tree

In [5]:
%%time
full_dt = DecisionTreeRegressor(criterion="mae")
full_dt.fit(X_train, y_train)

print(f"Tree Depth  = {full_dt.get_depth()}")
print(f"Tree LEaves = {full_dt.get_n_leaves()}")

Tree Depth  = 26
Tree LEaves = 1374
Wall time: 769 ms


In [6]:
full_dt_feature_imp = pd.Series(full_dt.feature_importances_, index=X_train.columns).sort_values(ascending=False).round(4) * 100; full_dt_feature_imp.head(10)

Total_SF        43.15
OverallQual     10.28
Neighborhood    10.00
GrLivArea        4.54
BsmtFinSF1       2.23
Remodel_Age      2.23
GarageArea       1.85
LotFrontage      1.82
LotArea          1.77
MoSold           1.24
dtype: float64

In [7]:
ccpp = pd.DataFrame(full_dt.cost_complexity_pruning_path(X_train, y_train))

In [8]:
ccpp

ccp_alphas    impurities
0         0.000000      0.000000
1         0.010274      0.010274
2         0.034247      0.044521
3         0.034247      0.078767
4         0.041781      0.120548
...            ...           ...
1267   1942.673973  29345.836301
1268   2023.165068  31369.001370
1269   3513.057534  34882.058904
1270   5238.838356  40120.897260
1271  15412.758904  55533.656164

[1272 rows x 2 columns]

In [11]:
%%time
param_distributions = {
    "criterion": ['mae'],
    "ccp_alpha": ccpp.ccp_alphas.values
}

dt_rscv = RandomizedSearchCV(
    DecisionTreeRegressor(),
    param_distributions,
    n_iter       = 100,
    scoring      = "neg_mean_absolute_error",
    n_jobs       = -2,
    cv           = cv,
    verbose      = 1,
    random_state = 7013,
)
dt_rscv.fit(X_train, y_train)

Fitting 50 folds for each of 100 candidates, totalling 5000 fits
Wall time: 23min 44s


RandomizedSearchCV(cv=RepeatedKFold(n_repeats=10, n_splits=5, random_state=3007),
                   estimator=DecisionTreeRegressor(), n_iter=100, n_jobs=-2,
                   param_distributions={'ccp_alpha': array([0.00000000e+00, 1.02739726e-02, 3.42465753e-02, ...,
       3.51305753e+03, 5.23883836e+03, 1.54127589e+04]),
                                        'criterion': ['mae']},
                   random_state=7013, scoring='neg_mean_absolute_error',
                   verbose=1)

In [12]:
pd.DataFrame(dt_rscv.cv_results_).sort_values(by="rank_test_score").head(5)

mean_fit_time  std_fit_time  mean_score_time  std_score_time  \
64       0.745176      0.060627         0.005962        0.002642   
72       0.958833      0.235419         0.008358        0.009308   
25       0.852104      0.053867         0.006354        0.002674   
95       1.032127      0.247019         0.008588        0.006447   
37       0.771214      0.085388         0.006229        0.003680   

   param_criterion param_ccp_alpha  \
64             mae       69.182192   
72             mae       80.821918   
25             mae       64.641096   
95             mae       54.547945   
37             mae       54.863014   

                                               params  split0_test_score  \
64  {'criterion': 'mae', 'ccp_alpha': 69.182191780...      -22838.717466   
72  {'criterion': 'mae', 'ccp_alpha': 80.821917808...      -24265.217466   
25  {'criterion': 'mae', 'ccp_alpha': 64.641095890...      -23645.703767   
95  {'criterion': 'mae', 'ccp_alpha': 54.547945205...      -23915.590753   
37  {'criterion': 'mae', 'ccp_alpha': 54.863013698...      -24378.243151   

    split1_test_score  split2_test_score  split3_test_score  \
64      -24137.402397      -24960.195205      -25154.340753   
72      -23718.717466      -23205.351027      -25202.063356   
25      -24693.308219      -25172.224315      -25956.902397   
95      -24208.465753      -23360.071918      -26010.583904   
37      -24325.941781      -23780.539384      -24964.693493   

    split4_test_score  split5_test_score  split6_test_score  \
64      -22147.013699      -20310.340753      -21906.272260   
72      -22074.000000      -19572.751712      -21449.587329   
25      -22677.150685      -19817.520548      -20970.142123   
95      -22847.469178      -19421.719178      -20787.371575   
37      -22577.373288      -20260.695205      -21059.118151   

    split7_test_score  split8_test_score  split9_test_score  \
64      -25179.222603      -22491.970890      -24736.517123   
72      -23871.513699      -22995.738014      -25646.226027   
25      -24137.671233      -22604.842466      -25594.726027   
95      -23782.962329      -23191.614726      -24879.212329   
37      -24128.750000      -22196.950342      -25137.318493   

    split10_test_score  split11_test_score  split12_test_score  \
64       -24055.712329       -23485.041096       -23565.191781   
72       -24483.914384       -22214.893836       -22857.366438   
25       -24373.410959       -22718.638699       -22854.476027   
95       -23289.361301       -22622.484589       -24027.630137   
37       -23753.744863       -22943.982877       -24079.856164   

    split13_test_score  split14_test_score  split15_test_score  \
64       -21578.708904       -21965.714041       -23060.851027   
72       -21210.104452       -23248.232877       -22775.511986   
25       -22025.693493       -23081.642123       -21707.412671   
95       -22507.583904       -23459.922945       -24152.820205   
37       -21676.469178       -22635.313356       -23946.895548   

    split16_test_score  split17_test_score  split18_test_score  \
64       -20001.698630       -24169.121575       -21837.967466   
72       -19458.638699       -23906.011986       -21788.183219   
25       -20919.477740       -24360.626712       -22309.143836   
95       -20001.106164       -23758.056507       -21336.107877   
37       -19518.479452       -23736.025685       -22266.611301   

    split19_test_score  split20_test_score  split21_test_score  \
64       -23645.618151       -22708.224315       -21910.583904   
72       -22790.602740       -23338.017123       -22570.547945   
25       -22692.241438       -23164.428082       -22068.464041   
95       -22880.232877       -22882.785959       -22358.393836   
37       -22346.803082       -23286.638699       -22690.054795   

    split22_test_score  split23_test_score  split24_test_score  \
64       -21061.416096       -22740.299658       -23344.960616   
72       -21633.936644       -22956.297945       -22